In [63]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
import cv2

In [2]:
def scaled(img, scale=20000):
    """First we find really high values, which are the bright spots and turn them down
    """
    dtype = img.dtype    
    epsilon = 0.99    
    _max = np.quantile(img[img>0], epsilon)
    scaled = (img * (scale / _max)).astype(dtype) # scale the image from original values to e.g., 30000/10000
    del img
    return scaled

def equalized(fixed, cliplimit=2):
    """Takes an image that has already been scaled and uses opencv adaptive histogram
    equalization. This cases uses 5 as the clip limit and splits the image into rows
    and columns. A higher cliplimit will make the image brighter. A cliplimit of 1 will
    do nothing. 

    :param fixed: image we are working on
    :return: a better looking image
    """
    
    clahe = cv2.createCLAHE(clipLimit=cliplimit, tileGridSize=(8, 8))
    fixed = clahe.apply(fixed)
    return fixed

In [ ]:
animal = 'DK101'
DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/tif'
filepath = os.path.join(DIR, 'DK101_slide033_2023_02_15_axion2_S1_C1.tif')

In [ ]:
#For bigarr, look at 15812x43685, 16816x44463
startrow = 43685
endrow = 44463
startcol = 15812
endcol = 16816

In [ ]:
bigarr = io.imread(filepath)
print(f'dtype={bigarr.dtype} shape={bigarr.shape}')

In [ ]:
%%time
#scale = 2**16 - 1
scale = 20000
bigarr_scaled = scaled(bigarr, scale)
print(f'dtype={bigarr_scaled.dtype} shape={bigarr_scaled.shape}')
cropped = bigarr_scaled[startrow:endrow, startcol:endcol]
print(f'dtype={cropped.dtype} shape={cropped.shape}')
outpath = os.path.join(DIR, 'cropped_test.tif')
io.imsave(outpath, cropped)

In [ ]:
scale = 2**16 - 1
cropped_scaled = scaled(cropped, scale)
outpath = os.path.join(DIR, f'cropped_scaled_{scale}.tif')
if not os.path.exists(outpath):
    io.imsave(outpath, cropped_scaled)